# Tutorial for CombinedTree and CombinedForest
## 1. Introduction
### 1.1 What is a CombinedTree?
A **CombinedTree** is an individual tree-based model that follows a _manually specified_ computation rule (a formula) but is split into multiple parts (sub-trees). Each part is an independent tree evolved by the genetic algorithm. For example, if you have:

```python
formula = lambda A, B, C: A + B * C
```

- `A`, `B`, and `C` are considered separate sub-trees, each evolved independently.
- The final output of the CombinedTree is computed by substituting the outputs of sub-trees `A`, `B`, `C` into the formula (`A + B * C`).

### 1.2 What is a CombinedForest?
A **CombinedForest** is simply a _population_ of CombinedTrees. Since EvoGP is a population-based genetic programming framework, we often manipulate and evaluate populations (forests) as the central objects rather than individual solutions (trees). 

Thus, practically, we usually:
1. Define a **CombinedForest** which internally maintains multiple sub-forests (one for each parameter).
2. Evolve it via the usual genetic operators (selection, crossover, mutation) in EvoGP.


## 2. How to Create a CombinedForest

When creating a **CombinedForest**, there are three important elements to provide:

1. **Formula** (the computation rule).
2. **Descriptors** (the blueprint that controls how each sub-tree is generated and mutated).
3. **Population size** (`pop_size`).
4. **share_input** (Whether all patterns in the formula share the same input, currently it **MUST be True**)

### 2.1 Defining a Formula
A formula must be a Python callable (e.g., `lambda`) that:
- Has _at least one_ parameter.  
- Uses only supported operations (e.g., `+`, `-`, `*`, `/`, or certain PyTorch math functions like `torch.pow`, `torch.tan`, etc.) to combine the parameters.  

For example:
```python
formula = lambda A, B, C: A + B * C
```
or 
```python
formula = lambda A, B: torch.pow(A, B)
```


### 2.2 Descriptors
**Descriptors** in EvoGP define how trees (individuals) are generated and mutated. Each parameter in your formula corresponds to one descriptor.

- **Single Descriptor**: You can pass one `GenerateDescriptor` object if you want _all parameters_ (`A`, `B`, `C`, ...) to be generated with the same rules.
- **List of Descriptors**: If you need different rules for each parameter, supply a list of descriptors, one per formula parameter.

Check out `evogp_intro.ipynb` or the main EvoGP tutorials for details on `GenerateDescriptor`. Here is a reminder of what it might look like:


In [4]:
from evogp.tree import CombinedForest, GenerateDescriptor
descriptor = GenerateDescriptor(
    max_tree_len=16,
    input_len=3,
    output_len=1,
    using_funcs=["+", "-", "*", "/", "neg"],  # Allowed function set
    max_layer_cnt=4,
    const_samples=[0, 0.5, 1]  # constant choices
)

## 3. Evolving a CombinedForest with an Algorithm

In EvoGP, after you create a **CombinedForest**, you typically feed it into a genetic algorithm pipeline. This pipeline handles:
1. **Selection**: how individuals are chosen for breeding (or carried forward as elites). 
2. **Crossover**: how two individuals exchange parts of their trees.
3. **Mutation**: how individuals are altered (sub-trees replaced, etc.).

You can just use the followings:

In [5]:
from evogp.algorithm import (
    DefaultSelection,
    CombinedDefaultMutation,
    CombinedDefaultCrossover,
)
crossover=CombinedDefaultCrossover()
mutation=CombinedDefaultMutation(
    mutation_rate=0.2, descriptors=descriptor
)
selection=DefaultSelection(survival_rate=0.3, elite_rate=0.01)

## 4. Converting a CombinedTree (or CombinedForest) to Sympy Expressions

Once you have evolved a best solution (or a set of solutions), you can:
- Access each parameter sub-tree directly (e.g., `best.A`, `best.B`, `best.C`).
- Convert these sub-trees into **sympy expressions** by calling `.to_sympy_expr()`.
- Convert the **entire** CombinedTree into a Sympy expression using `combined_tree.to_sympy_expr(SYMPY_FORMULA)`, which will apply your top-level formula to each sub-tree's Sympy form.

This is helpful for interpretability (e.g., to see an analytic formula of your best solution).

## Complete Example

In [7]:
# import nessary packages
import torch
from evogp.tree import CombinedForest, GenerateDescriptor
from evogp.algorithm import (
    GeneticProgramming,
    DefaultSelection,
    CombinedDefaultMutation,
    CombinedDefaultCrossover,
)
from evogp.problem import SymbolicRegression
from evogp.pipeline import StandardPipeline

In [9]:
# Prepare sample XOR data (running on CUDA for speed, but CPU is fine)
XOR_INPUTS = torch.tensor(
    [
        [0, 0, 0],
        [0, 0, 1],
        [0, 1, 0],
        [0, 1, 1],
        [1, 0, 0],
        [1, 0, 1],
        [1, 1, 0],
        [1, 1, 1],
    ],
    dtype=torch.float,
    device="cuda",
)

XOR_OUTPUTS = torch.tensor(
    [[0], [1], [1], [0], [1], [0], [0], [1]],
    dtype=torch.float,
    device="cuda",
)

# Define a symbolic regression problem
problem = SymbolicRegression(
    datapoints=XOR_INPUTS, 
    labels=XOR_OUTPUTS, 
    execute_mode="torch"
)

In [12]:
# Create a descriptor for generating sub-trees
descriptor = GenerateDescriptor(
    max_tree_len=16,
    input_len=problem.problem_dim,   # 3 inputs for the XOR
    output_len=problem.solution_dim, # 1 output
    using_funcs=["+", "-", "*", "/"],
    max_layer_cnt=4,
    const_samples=[0, 0.5, 1]
)

# Create the initial CombinedForest with a formula. 
# In this example, we use: formula=lambda A, B, C: A + B * C
initial_population = CombinedForest.random_generate(
    pop_size=5000,
    formula=lambda A, B, C: A + B * C,
    descriptors=descriptor,
    share_input=True,  # same inputs for both sub-forests
)

In [13]:
# Set up our GeneticProgramming algorithm with default operators
algorithm = GeneticProgramming(
    initial_forest=initial_population,
    crossover=CombinedDefaultCrossover(),
    mutation=CombinedDefaultMutation(
        mutation_rate=0.2, 
        descriptors=descriptor.update(max_layer_cnt=3)
    ),
    selection=DefaultSelection(survival_rate=0.3, elite_rate=0.01),
)

In [14]:
# We wrap it in a StandardPipeline for convenience:
pipeline = StandardPipeline(
    algorithm,
    problem,
    generation_limit=100,
)

# Run the pipeline and get the best solution
best = pipeline.run()

Generation: 0, Cost time: 155.16ms
 	fitness: valid cnt: 781, max: -0.2500, min: -185.5625, mean: -3.1468, std: 9.3311

Generation: 1, Cost time: 6.00ms
 	fitness: valid cnt: 2272, max: -0.2500, min: -475.5625, mean: -3.0654, std: 12.6313

Generation: 2, Cost time: 5.00ms
 	fitness: valid cnt: 4384, max: -0.2188, min: -47.5000, mean: -1.2731, std: 1.7615

Generation: 3, Cost time: 5.00ms
 	fitness: valid cnt: 4486, max: -0.1875, min: -64.5000, mean: -0.8127, std: 1.5118

Generation: 4, Cost time: 5.00ms
 	fitness: valid cnt: 4519, max: -0.1250, min: -19.3750, mean: -0.6665, std: 0.8676

Generation: 5, Cost time: 4.38ms
 	fitness: valid cnt: 4543, max: -0.1250, min: -20.6875, mean: -0.5460, std: 0.7388

Generation: 6, Cost time: 6.00ms
 	fitness: valid cnt: 4545, max: -0.1250, min: -17.3750, mean: -0.4322, std: 0.5855

Generation: 7, Cost time: 5.00ms
 	fitness: valid cnt: 4586, max: -0.1250, min: -100.2500, mean: -0.4729, std: 1.8531

Generation: 8, Cost time: 5.00ms
 	fitness: valid c

In [15]:
# Evaluate the best solution on the XOR inputs:
pred_res = best.forward(XOR_INPUTS)
print("Predictions from best:", pred_res)

Predictions from best: tensor([[0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.]], device='cuda:0')


In [19]:
# Check the best solution:

# Check each parts of the solution:
best.A.to_sympy_expr()  # use "A" because we named it in the formula

0.5

In [20]:
print(best.B.to_sympy_expr())
print(best.C.to_sympy_expr())

1.0
1.0*(x0 - 0.5)*(x1 - 0.5)/(x2 - 0.5)


In [21]:
# Check the whole solution:
best.to_sympy_expr(lambda A, B, C: A + B * C)

1.0*(x0 - 0.5)*(x1 - 0.5)/(x2 - 0.5) + 0.5